In [12]:
import pandas as pd
import itables
from itables import downsample, options, sample_dfs
import itables.typing as typing


# Utilise le bon format de chemin (slashs ou chaîne brute)
df = pd.read_csv('data/SpotifyFeatures.csv')



# === 1. Charger le fichier CSV ===


# === 2. Nettoyage du nom d'artiste pour éviter les problèmes d'espaces/casse/accents ===
df['artist_name'] = df['artist_name'].str.strip().str.casefold()

# === 3. Liste normalisée des artistes variété française ===
artistes_variete_francaise = [
    'jean-jacques goldman', 'michel sardou', 'francis cabrel', 'julien clerc',
    'claude françois', 'patrick bruel', 'michel delpech', 'véronique sanson',
    'joe dassin', 'charles aznavour', 'édith piaf', 'serge lama',
    'michel polnareff', 'daniel balavoine', 'alain souchon', 'laurent voulzy',
    'françoise hardy', 'gilbert bécaud', 'barbara', 'charles trenet',
    'hélène ségara', 'lââm', 'karen cheryl', 'emji', 'toma', 'georges brassens',
    'jacques brel', 'léo ferré', 'juliette gréco', 'boris vian', 'serge gainsbourg',
    'jean ferrat', 'yves montand', 'renaud', 'maxime le forestier', 'france gall',
    'michel berger', 'zazie', 'calogero', 'benabar', 'vincent delerm', 'zaz',
    'stromae', 'camille', 'christine and the queens', 'clara luciani', 'julien doré',
    'vianney', 'louane', 'hoshi', 'pierre de maere', 'pomme', 'lomepal',
    'gaël faye', 'eddy de pretto', 'terrenoire'
]

# === 4. Filtrage des chansons dont l’artiste est dans cette liste ===
df_variete_fr = df[df['artist_name'].isin(artistes_variete_francaise)]

# === 5. Suppression des doublons par track_id ===
df_variete_fr = df_variete_fr.drop_duplicates(subset='track_id')

# === 6. Résultat final ===
print(f"{len(df_variete_fr)} chansons de variété française extraites.")
df_variete_fr[['track_name', 'artist_name']]

73 chansons de variété française extraites.


,track_name,artist_name
57549,Femme,hélène ségara
113731,Papaoutai,stromae
132778,Les copains d'abord,georges brassens
134220,Je Me Suis Fait Tout Petit,georges brassens
134513,Chanson pour l'auvergnat,georges brassens
...,...,...
195042,Tu ne seras jamais libre,hélène ségara
195124,O sole mio,hélène ségara
195138,Quand l'éternité,hélène ségara
195158,J'attends,hélène ségara
